<a href="https://colab.research.google.com/github/finiteautomata/hatespeech-classification/blob/main/notebooks/Hatespeech_Colab_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de modelos de Hate Speech usando TPUs

En esta notebook haremos el entrenamiento de un modelo "pesado" de Hate Speech. La idea es usar las TPU para poder entrenar un BERT que toma ~ 10 hs en dos GPU

Ahora, a montar Drive

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly
!python pytorch-xla-env-setup.py --version 1.8.1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  73085      0 --:--:-- --:--:-- --:--:-- 72056
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-1.8.1 ...
     |████████████████████████████████| 61kB 3.1MB/s 
Uninstalling torch-1.8.1+cu101:
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
Done updating TPU runtime
  Successfully uninstalled torch-1.8.1+cu101
Uninstalling torchvision-0.9.1+cu101:
  Successfully uninstalled torchvision-0.9.1+cu101
Copying gs://tpu-pytorch/wheels/torch-1.8.1-cp37-cp37m-linux_x86_64.whl...
- [1 files][126.5 MiB/126.

In [2]:
!pip freeze | grep torch

torch==1.8.0a0+56b43f4
torch-xla==1.8.1
torchsummary==1.5.1
torchtext==0.9.1
torchvision==0.10.0a0+3926c90


In [3]:
from google.colab import drive

drive.mount("/content/drive/")

Mounted at /content/drive/


Al toque perrito

In [10]:
repo_dir = "/content/drive/Shareddrives/HateSpeech/hatespeech-classification"
!cd $repo_dir && git pull
!cd $repo_dir && pip install -e . 
!pip install emoji pandarallel absl-py

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/finiteautomata/hatespeech-classification
   a6e9316..ade9e70  main       -> origin/main
Updating a6e9316..ade9e70
Fast-forward
 hatedetection/training.py | 6 +++---
 1 file changed, 3 insertions(+), 3 deletions(-)
Obtaining file:///content/drive/Shareddrives/HateSpeech/hatespeech-classification
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Found existing installation: hatespeech-classification 0.0.1
    Can't uninstall 'hatespeech-classification'. No files were found to uninstall.
  Running setup.py develop for hatespeech-classification


### NOTA IMPORTANTE

Reiniciar notebook después de correr esto. Por algún motivo extraño, no reconoce el paquete que acabamos de instalar localmente

In [1]:
from hatedetection.metrics import compute_hate_metrics
from hatedetection.preprocessing import special_tokens
from hatedetection.training import load_hatespeech_model_and_tokenizer, tokenize
from transformers import (
    BertTokenizerFast, TrainingArguments, Trainer,
    AutoModelForSequenceClassification, AutoTokenizer
)

def train_hatespeech_classifier(
    model, train_dataset, dev_dataset,
    max_length=None, epochs=10, warmup_proportion=0.1,
    ):
    per_device_train_batch_size = 16
    num_cores = 8
    batch_size = num_cores * per_device_train_batch_size

    total_steps = (epochs * len(train_dataset)) // batch_size
    warmup_steps = int(warmup_proportion * total_steps)
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=epochs,
        warmup_steps=warmup_steps,
        evaluation_strategy="epoch",
        per_device_train_batch_size=per_device_train_batch_size,
        weight_decay=0.01,
        logging_dir='./logs',
        eval_accumulation_steps=1,
        #load_best_model_at_end=True,
        #metric_for_best_model="f1",
    )

    results = []

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_hate_metrics,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
    )

    trainer.place_model_on_device = False

    trainer.train()

    return trainer


In [2]:
from hatedetection import load_datasets

model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
context = 'title+body'

"""
Todo esto es para evitar que la memoria salte por los aires por usar 'fork' en XLA
Ver: https://pytorch.org/xla/release/1.8/index.html#torch_xla.distributed.xla_multiprocessing.MpModelWrapper

"""
print("Cargando modelo")

model, tokenizer = load_hatespeech_model_and_tokenizer(model_name, context)



"""
TODO: ver si podemos salvarlo primero

https://huggingface.co/docs/datasets/processing.html#saving-a-processed-dataset-on-disk-and-reload-it
"""

add_body = True if 'body' in context else False
train_dataset, dev_dataset, _ = load_datasets(add_body=add_body)

tokenize_fun = lambda batch: tokenize(tokenizer, batch, context=context)
train_dataset = train_dataset.map(tokenize_fun, batched=True, batch_size=64)
dev_dataset = dev_dataset.map(tokenize_fun, batched=True, batch_size=64)


def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['HATEFUL']})
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)

Cargando modelo


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

Los salvamos a disco. Esto es porque tarda un huevo si lo hacemos 8 veces.

In [3]:
train_path = "datasets/train/"
dev_path = "datasets/dev/"

train_dataset.save_to_disk(train_path)
dev_dataset.save_to_disk(dev_path)

In [7]:
import os
import torch
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from datasets import load_from_disk
import sys


"""
Todo esto es para evitar que la memoria salte por los aires por usar 'fork' en XLA
Ver: https://pytorch.org/xla/release/1.8/index.html#torch_xla.distributed.xla_multiprocessing.MpModelWrapper

"""

WRAPPED_MODEL = xmp.MpModelWrapper(model)


def train_model(
    model, tokenizer, output_path, context='none',
    batch_size=8, eval_batch_size=8, max_length=None, slice=None,
    epochs=10, warmup_proportion=0.1,
    ):
    """
    """ 
    print("*"*80)
    print("Training hate speech classifier")



    print("Loading datasets... ", end="")

    train_dataset = load_from_disk(train_path)
    dev_dataset = load_from_disk(dev_path)

    if slice:
        print("Slicing")
        train_dataset = train_dataset.select(range(slice))
        dev_dataset = dev_dataset.select(range(slice))
    print("Done")


    trainer = train_hatespeech_classifier(
        model, train_dataset=train_dataset, dev_dataset=dev_dataset,
        epochs=epochs, warmup_proportion=warmup_proportion, 
    )
    print("Terminamos de entrenar perrito!")
    print("\n"*3, "Saving...")
    trainer.save_model(output_path)
    tokenizer.save_pretrained(output_path)

    print(f"Models saved at {output_path}")



In [8]:
import os
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

output_path = "/content/drive/Shareddrives/HateSpeech/models/bert-title-body-hate-speech-es/"

def _mp_fn(index):
    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)


    train_model(
        model, tokenizer, epochs=10, output_path=output_path,  
        #slice=2048,
    )

xmp.spawn(_mp_fn, start_method="fork")

********************************************************************************
Training hate speech classifier
Loading datasets... Done
********************************************************************************
Training hate speech classifier
Loading datasets... Done


Epoch,Training Loss,Validation Loss,Accuracy,F1,Macro F1,Precision,Recall,Runtime,Samples Per Second
1,No log,0.265763,0.898858,0.649410,0.795157,0.687903,0.614996,87.226100,104.395000
2,0.282300,0.295350,0.889523,0.675693,0.804557,0.611079,0.755588,66.890800,136.132000
3,0.282300,0.364041,0.920163,0.703144,0.828512,0.810734,0.620764,66.608700,136.709000
4,0.123900,0.414487,0.920163,0.732425,0.842754,0.748120,0.717376,67.257100,135.391000
5,0.123900,0.498752,0.915660,0.722142,0.836213,0.724764,0.719539,67.722100,134.461000
6,0.048100,0.575872,0.921041,0.730004,0.841881,0.761755,0.700793,71.839600,126.755000
7,0.048100,0.604864,0.923347,0.734196,0.844706,0.778047,0.695025,67.575000,134.754000
8,0.013900,0.650868,0.924555,0.735056,0.845536,0.790216,0.687094,67.734800,134.436000
9,0.005800,0.662315,0.924336,0.738718,0.847240,0.779200,0.702235,67.418400,135.067000
10,0.005800,0.675427,0.923237,0.738300,0.846661,0.767913,0.710887,67.127100,135.653000


Epoch,Training Loss,Validation Loss,Accuracy,F1,Macro F1,Precision,Recall,Runtime,Samples Per Second
1,No log,0.265763,0.898858,0.649410,0.795157,0.687903,0.614996,87.251700,104.365000
2,0.293700,0.295350,0.889523,0.675693,0.804557,0.611079,0.755588,66.866700,136.181000
3,0.293700,0.364041,0.920163,0.703144,0.828512,0.810734,0.620764,66.595200,136.737000
4,0.117600,0.414487,0.920163,0.732425,0.842754,0.748120,0.717376,67.254200,135.397000
5,0.117600,0.498752,0.915660,0.722142,0.836213,0.724764,0.719539,67.684900,134.535000
6,0.046800,0.575872,0.921041,0.730004,0.841881,0.761755,0.700793,72.084200,126.325000
7,0.046800,0.604864,0.923347,0.734196,0.844706,0.778047,0.695025,67.570400,134.763000
8,0.016600,0.650868,0.924555,0.735056,0.845536,0.790216,0.687094,67.751100,134.404000
9,0.006600,0.662315,0.924336,0.738718,0.847240,0.779200,0.702235,67.346100,135.212000
10,0.006600,0.675427,0.923237,0.738300,0.846661,0.767913,0.710887,67.355600,135.193000


********************************************************************************
Training hate speech classifier
Loading datasets... Done


Epoch,Training Loss,Validation Loss,Accuracy,F1,Macro F1,Precision,Recall,Runtime,Samples Per Second
1,No log,0.265763,0.898858,0.649410,0.795157,0.687903,0.614996,87.246900,104.370000
2,0.286400,0.295350,0.889523,0.675693,0.804557,0.611079,0.755588,66.869800,136.175000
3,0.286400,0.364041,0.920163,0.703144,0.828512,0.810734,0.620764,66.594700,136.738000
4,0.113300,0.414487,0.920163,0.732425,0.842754,0.748120,0.717376,67.272800,135.359000
5,0.113300,0.498752,0.915660,0.722142,0.836213,0.724764,0.719539,67.693100,134.519000
6,0.045700,0.575872,0.921041,0.730004,0.841881,0.761755,0.700793,72.062600,126.362000
7,0.045700,0.604864,0.923347,0.734196,0.844706,0.778047,0.695025,67.518500,134.867000
8,0.020800,0.650868,0.924555,0.735056,0.845536,0.790216,0.687094,67.726300,134.453000
9,0.006900,0.662315,0.924336,0.738718,0.847240,0.779200,0.702235,67.412600,135.079000
10,0.006900,0.675427,0.923237,0.738300,0.846661,0.767913,0.710887,67.332000,135.240000


...

********************************************************************************
Training hate speech classifier
Loading datasets... Done


Epoch,Training Loss,Validation Loss,Accuracy,F1,Macro F1,Precision,Recall,Runtime,Samples Per Second
1,No log,0.265763,0.898858,0.649410,0.795157,0.687903,0.614996,87.238300,104.381000
2,0.281500,0.295350,0.889523,0.675693,0.804557,0.611079,0.755588,66.813700,136.289000
3,0.281500,0.364041,0.920163,0.703144,0.828512,0.810734,0.620764,66.557000,136.815000
4,0.137100,0.414487,0.920163,0.732425,0.842754,0.748120,0.717376,67.248900,135.407000
5,0.137100,0.498752,0.915660,0.722142,0.836213,0.724764,0.719539,67.686000,134.533000
6,0.051800,0.575872,0.921041,0.730004,0.841881,0.761755,0.700793,71.821300,126.787000
7,0.051800,0.604864,0.923347,0.734196,0.844706,0.778047,0.695025,67.545300,134.813000
8,0.012700,0.650868,0.924555,0.735056,0.845536,0.790216,0.687094,67.728700,134.448000
9,0.008400,0.662315,0.924336,0.738718,0.847240,0.779200,0.702235,67.417800,135.068000
10,0.008400,0.675427,0.923237,0.738300,0.846661,0.767913,0.710887,67.173300,135.560000


********************************************************************************
Training hate speech classifier
Loading datasets... Done


Epoch,Training Loss,Validation Loss,Accuracy,F1,Macro F1,Precision,Recall,Runtime,Samples Per Second
1,No log,0.265763,0.898858,0.649410,0.795157,0.687903,0.614996,87.239900,104.379000
2,0.276500,0.295350,0.889523,0.675693,0.804557,0.611079,0.755588,66.899300,136.115000
3,0.276500,0.364041,0.920163,0.703144,0.828512,0.810734,0.620764,66.577000,136.774000
4,0.136300,0.414487,0.920163,0.732425,0.842754,0.748120,0.717376,67.262900,135.379000
5,0.136300,0.498752,0.915660,0.722142,0.836213,0.724764,0.719539,67.646900,134.611000
6,0.048100,0.575872,0.921041,0.730004,0.841881,0.761755,0.700793,71.866300,126.708000
7,0.048100,0.604864,0.923347,0.734196,0.844706,0.778047,0.695025,67.554600,134.795000
8,0.015400,0.650868,0.924555,0.735056,0.845536,0.790216,0.687094,67.682300,134.540000
9,0.007000,0.662315,0.924336,0.738718,0.847240,0.779200,0.702235,67.418400,135.067000
10,0.007000,0.675427,0.923237,0.738300,0.846661,0.767913,0.710887,67.205900,135.494000


Terminamos de entrenar perrito!
Terminamos de entrenar perrito!
Terminamos de entrenar perrito!






 Saving...



Terminamos de entrenar perrito!
 Saving...



 Saving...
 Saving...
Terminamos de entrenar perrito!



 Saving...
Terminamos de entrenar perrito!



 Saving...
Terminamos de entrenar perrito!



 Saving...
Terminamos de entrenar perrito!



 Saving...
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-title-body-hate-speech-es/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-title-body-hate-speech-es/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-title-body-hate-speech-es/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-title-body-hate-speech-es/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-title-body-hate-speech-es/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-title-body-hate-speech-es/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-t

In [6]:
!sudo dmesg -T

[Tue May  4 12:01:34 2021] Linux version 4.19.112+ (builder@a12462ca91c8) (Chromium OS 10.0_pre377782_p20200113-r10 clang version 10.0.0 (/var/cache/chromeos-cache/distfiles/host/egit-src/llvm-project 4e8231b5cf0f5f62c7a51a857e29f5be5cb55734)) #1 SMP Thu Jul 23 08:00:38 PDT 2020
[Tue May  4 12:01:34 2021] Command line: BOOT_IMAGE=/syslinux/vmlinuz.A init=/usr/lib/systemd/systemd boot=local rootwait ro noresume noswap loglevel=7 noinitrd console=ttyS0 security=apparmor virtio_net.napi_tx=1 systemd.unified_cgroup_hierarchy=false systemd.legacy_systemd_cgroup_controller=false csm.disabled=1 dm_verity.error_behavior=3 dm_verity.max_bios=-1 dm_verity.dev_wait=1 i915.modeset=1 cros_efi root=/dev/dm-0 "dm=1 vroot none ro 1,0 4077568 verity payload=PARTUUID=555BDB75-CBD7-CD4A-B24E-29B13D7AC0DF hashtree=PARTUUID=555BDB75-CBD7-CD4A-B24E-29B13D7AC0DF hashstart=4077568 alg=sha256 root_hexdigest=42104d547ac104fb7061529e78f53e4f3e8c3d3cbb040dc6e0f84aad68491347 salt=9dc7f3acc4e2ce65be16356e960c2b21b5